## mzML Format test

In [ ]:
import base64, struct

In [ ]:
def base64_to_float(in_base64_str, float_bit=64):
    dec_str = base64.decodebytes(in_base64_str)
    
    if(float_bit==64):
        dec_float = struct.unpack('<%sd' % (len(dec_str) // 8), dec_str) # for 64 bit data
    else:
        dec_float = struct.unpack('<%sf' % (len(dec_str) // 4), dec_str) # for 32 bit data
    
    return dec_float

In [ ]:
mz_enc = b"AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAAAAAAAQQAAAAAAAABRAAAAAAAAAGEAAAAAAAAAcQAAAAAAAACBAAAAAAAAAIkA="
intensity_enc = b"AAAAAAAAJEAAAAAAAAAiQAAAAAAAACBAAAAAAAAAHEAAAAAAAAAYQAAAAAAAABRAAAAAAAAAEEAAAAAAAAAIQAAAAAAAAABAAAAAAAAA8D8="

mz = base64_to_float(mz_enc)
print(mz)

intensity = base64_to_float(intensity_enc)
print(intensity)

In [ ]:
mz_enc = b"AAAAAAAAAAAAAAAAAADwPwAAAAAAAABAAAAAAAAACEAAAAAAAAAQQAAAAAAAABRAAAAAAAAAGEAAAAAAAAAcQAAAAAAAACBAAAAAAAAAIkAAAAAAAAAkQAAAAAAAACZAAAAAAAAAKEAAAAAAAAAqQAAAAAAAACxAAAAAAAAALkAAAAAAAAAwQAAAAAAAADFAAAAAAAAAMkAAAAAAAAAzQA=="
intensity_enc = b"AAAAAAAANEAAAAAAAAAzQAAAAAAAADJAAAAAAAAAMUAAAAAAAAAwQAAAAAAAAC5AAAAAAAAALEAAAAAAAAAqQAAAAAAAAChAAAAAAAAAJkAAAAAAAAAkQAAAAAAAACJAAAAAAAAAIEAAAAAAAAAcQAAAAAAAABhAAAAAAAAAFEAAAAAAAAAQQAAAAAAAAAhAAAAAAAAAAEAAAAAAAADwPw=="

mz = base64_to_float(mz_enc)
print(mz)

intensity = base64_to_float(intensity_enc)
print(intensity)

# MGF foramt analysis

In [7]:
import pyteomics.mgf
import os

In [8]:
mgf_path = '/home/workspace/MS-Tools/PXD001197/'

mgf_file = 'OrbiElite04549.mgf'

file_list = list()
file_list = file_list + [mgf_path + mgf_file]

In [ ]:
print(file_list)

In [ ]:
f_in = pyteomics.mgf.MGF(mgf_path+mgf_file)
filename = os.path.splitext(os.path.basename(f_in.name))[0]
for spectrum_i, spectrum_dict in enumerate(f_in):
    print(spectrum_i)

In [ ]:
print(spectrum_dict)

In [10]:
print(spectrum_dict['params'])

print(spectrum_dict['params']['title'])

print(spectrum_dict['params']['pepmass'][0])

print(spectrum_dict['params']['charge'])

# print(spectrum_dict['m/z array'])
# print(spectrum_dict['intensity array'])


{'title': 'controllerType=0 controllerNumber=1 scan=885', 'scans': '885', 'rtinseconds': 195.3685, 'pepmass': (730.829223632813, None), 'charge': [2]}
controllerType=0 controllerNumber=1 scan=885
730.829223632813
2+


In [ ]:
import numpy as np 
def test():
    quantization_error = np.zeros(2)
    quantization_error[0] = quantization_error[0] + 1
    quantization_error[1] = quantization_error[1] + 2
    return quantization_error

a = test()

In [16]:
a = {'b':1}
del a['b']

## Preprocessing for MGF files

In [26]:
import os
import sys
import json
import glob

import logging
import pyteomics.mgf
import numpy as np
from joblib import Parallel, delayed

logger = logging.getLogger('ms_preprocess_fp')

def run_baseline(**kargs):
    # Configure logging.
#     logging.captureWarnings(True)
#     root = logging.getLogger()
#     root.setLevel(logging.DEBUG)
#     handler = logging.StreamHandler(sys.stderr)
#     handler.setLevel(logging.DEBUG)
#     handler.setFormatter(logging.Formatter(
#         '{asctime} {levelname} [{name}/{processName}] {module}.{funcName} : '
#         '{message}', style='{'))
#     root.addHandler(handler)

#     print(kargs)
    
    ############### Preprocess spectra ###############
    logger.info("Start processsing spectra")
    results = Parallel(n_jobs=-1, verbose=0)(delayed(preprocess_spectrum)(file_i, **kargs) for file_i in kargs['file_list'])
    logger.info("End processsing spectra")
    
    ############### Merge and reduce results ###############
    logger.info("Merging spectra results")    
    merged_processed_spectra = {}
    total_quantization_error = np.zeros(2)

    spectra_len = np.array([])
    for result_i in results:
        ############### Gather spectra results ###############
        spec_i = result_i[0]
        spectra_len = np.append(spectra_len, result_i[2])
        if not merged_processed_spectra:
            merged_processed_spectra = spec_i
        else:
            for key in spec_i:
                if key not in merged_processed_spectra:
                    merged_processed_spectra[key] = spec_i[key]
                else:
                    merged_processed_spectra[key] = merged_processed_spectra[key] + spec_i[key]

        ############### Gather quantization error ###############
        quan_err_i = result_i[1]
        total_quantization_error = total_quantization_error+quan_err_i

    merged_processed_spectra['quantization_error'] = (total_quantization_error/np.sum(spectra_len)).tolist()
    logger.info("Merging spectra results complete!")
    
    # Dump preprocessed results
    logger.info("Exporting results")
    with open(kargs['out_file_name']+'_parms.json', 'w') as convert_file:
        kargs['fp_quantizer_mz'] = None
        kargs['fp_quantizer_peak'] = None
        convert_file.write(json.dumps(kargs))
    with open(kargs['out_file_name']+'_data.json', 'w') as convert_file:
        convert_file.write(json.dumps(merged_processed_spectra))
    
    mz_error = merged_processed_spectra['quantization_error'][0]
    peak_error = merged_processed_spectra['quantization_error'][1]
    logger.info("All finished! With quantization error: %f %f", mz_error, peak_error)
    
    return merged_processed_spectra, spectra_len
    
    
def preprocess_spectrum(mgf_file, **kargs):
    f_in = pyteomics.mgf.MGF(mgf_file)
    filename = os.path.splitext(os.path.basename(f_in.name))[0]

    logger.info('Processing mgf file: %s', filename)
    processed_spectra_list = {}
    quantization_error = np.zeros(2)
    spectra_len = np.array([])
    for spectrum_i, spectrum_dict in enumerate(f_in):
        spec_i_charge = str(spectrum_dict['params']['charge'])
        spec_i_title = spectrum_dict['params']['title']
        spec_i_precursor_mz = spectrum_dict['params']['pepmass'][0]

        spec_i_mz = spectrum_dict['m/z array'].astype(np.float64)
        spec_i_intensity = spectrum_dict['intensity array'].astype(np.float64)
        
        spectra_len = np.append(spectra_len, len(spec_i_mz))
#         print('maximum intensity: ', spec_i_intensity.max())
        if kargs['if_fp_quantize']:
            quantized_spec_i_mz = kargs['fp_quantizer_mz'].quantize(spec_i_mz)
            quantized_spec_i_intensity = kargs['fp_quantizer_peak'].quantize(spec_i_intensity)
            
            # 1. Filter spectrum out of mz range + low-intensity peaks
            spec_mask = filter_mz_intensity_range(quantized_spec_i_mz, quantized_spec_i_intensity, kargs['mz_min'], kargs['mz_max'], kargs['min_intensity'])
            (quantized_spec_i_mz , quantized_spec_i_intensity) = (quantized_spec_i_mz[spec_mask], quantized_spec_i_intensity[spec_mask])
            if not _check_spectrum_valid(quantized_spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                continue

            # 2. Remove peaks related to precursor
            if kargs['remove_precursor_tol'] is not None:
                spec_mask = _remove_precursor_peak(quantized_spec_i_mz, kargs['remove_precursor_tol'], spec_i_precursor_mz)
                (quantized_spec_i_mz , quantized_spec_i_intensity) = (quantized_spec_i_mz[spec_mask], quantized_spec_i_intensity[spec_mask])

                if not _check_spectrum_valid(quantized_spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                    continue

            # 3. Remove low-intensity peaks
            if kargs['max_peaks_used'] is not None:
                max_peaks_used = kargs['max_peaks_used']

                spec_mask = _select_top_intensity(quantized_spec_i_intensity, max_peaks_used)
                (quantized_spec_i_mz , quantized_spec_i_intensity) = (quantized_spec_i_mz[spec_mask], quantized_spec_i_intensity[spec_mask])

                if not _check_spectrum_valid(quantized_spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                    continue

            # 4. Sclae and normalize peaks
            quantized_spec_i_intensity = _scale_intensity(quantized_spec_i_intensity, kargs['scaling'])
            quantized_spec_i_intensity = _norm_intensity(quantized_spec_i_intensity)            
            

            
        # 1. Filter spectrum out of mz range + low-intensity peaks
        spec_mask = filter_mz_intensity_range(spec_i_mz, spec_i_intensity, kargs['mz_min'], kargs['mz_max'], kargs['min_intensity'])
        (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])
        if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
            continue

        # 2. Remove peaks related to precursor
        if kargs['remove_precursor_tol'] is not None:
            spec_mask = _remove_precursor_peak(spec_i_mz, kargs['remove_precursor_tol'], spec_i_precursor_mz)
            (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])

            if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                continue

        # 3. Remove low-intensity peaks
        if kargs['max_peaks_used'] is not None:
            max_peaks_used = kargs['max_peaks_used']

            spec_mask = _select_top_intensity(spec_i_intensity, max_peaks_used)
            (spec_i_mz , spec_i_intensity) = (spec_i_mz[spec_mask], spec_i_intensity[spec_mask])

            if not _check_spectrum_valid(spec_i_mz, kargs['min_peaks'], kargs['min_mz_range']):
                continue

        # 4. Sclae and normalize peaks
        spec_i_intensity = _scale_intensity(spec_i_intensity, kargs['scaling'])
        spec_i_intensity = _norm_intensity(spec_i_intensity)

        
        if kargs['if_fp_quantize']:
            if(len(quantized_spec_i_mz) < len(spec_i_mz)):
                temp_mz = np.zeros(spec_i_mz.shape)
                temp_intensity = np.zeros(spec_i_intensity.shape)
                
                temp_mz[:quantized_spec_i_mz.shape[0]] = quantized_spec_i_mz
                temp_intensity[:quantized_spec_i_intensity.shape[0]] = quantized_spec_i_intensity
                
                quantized_spec_i_mz = temp_mz
                quantized_spec_i_intensity = temp_intensity
                
            if(len(quantized_spec_i_mz) > len(spec_i_mz)):
                temp_mz = np.zeros(quantized_spec_i_mz.shape)
                temp_intensity = np.zeros(quantized_spec_i_intensity.shape)
                
                temp_mz[:spec_i_mz.shape[0]] = spec_i_mz
                temp_intensity[:spec_i_intensity.shape[0]] = spec_i_intensity
                
                spec_i_mz = temp_mz
                spec_i_intensity = temp_intensity

            quantization_error[0] = quantization_error[0] + np.absolute(np.subtract(quantized_spec_i_mz, spec_i_mz)).mean()   
            quantization_error[1] = quantization_error[1] + np.absolute(np.subtract(quantized_spec_i_intensity, spec_i_intensity)).mean()
        
        
        if spec_i_charge not in processed_spectra_list:
            processed_spectra_list[spec_i_charge] = []

        processed_spectra_list[spec_i_charge] = processed_spectra_list[spec_i_charge] + [spec_i_title]

    return processed_spectra_list, quantization_error, spectra_len


def filter_mz_intensity_range(spectrum_mz: np.ndarray, spectrum_intensity: np.ndarray, mz_min: float, mz_max: float, min_intensity: float = 0.0,):
    """
        Filter spectrum out of mz range and low-intensity peaks

        Parameters
        ----------
        spectrum_mz : np.ndarray
            M/z peaks of the cluster whose quality is checked.
        spectrum_intensity: np.ndarray
            Intensity of the cluster whose quality is checked.
        mz_min : float
            Minimum number of peaks the cluster has to contain.
        mz_max : float
            Minimum number of peaks the cluster has to contain.
        min_intensity : float
            Remove peaks whose intensity is below `min_intensity` percentage
            of the intensity of the most intense peak (the default is 0, which
            means that no minimum intensity filter will be applied).
        Returns
        -------
        np.ndarray
            Index mask specifying which peaks are retained after intensity filtering
        """
    mask_mz = (spectrum_mz >= mz_min) & (spectrum_mz <= mz_max)
    mask_intensity = spectrum_intensity >= min_intensity
    return mask_mz & mask_intensity

    
def _check_spectrum_valid(spectrum_mz: np.ndarray, min_peaks: int, min_mz_range: float) -> bool:
    """
    Check whether a cluster is of good enough quality to be used.
    Parameters
    ----------
    spectrum_mz : np.ndarray
        M/z peaks of the cluster whose quality is checked.
    min_peaks : int
        Minimum number of peaks the cluster has to contain.
    min_mz_range : float
        Minimum m/z range the cluster's peaks need to cover.
    Returns
    -------
    bool
        True if the cluster has enough peaks covering a wide enough mass
        range, False otherwise.
    """
    return (len(spectrum_mz) >= min_peaks and spectrum_mz[-1] - spectrum_mz[0] >= min_mz_range)   


def _remove_precursor_peak(spectrum_mz: np.ndarray, precursor_tolerance: float, precursor_mz: float):
    """
    Remove peaks realted to the precursor
    Parameters
    ----------
    spectrum_mz : np.ndarray
        M/z peaks of the cluster to be processed.
    precursor_tolerance : float
        Fragment mass tolerance around the precursor mass to remove the precursor peak.
    precursor_mz : float
        Precursor mz.
    Returns
    -------
    np.ndarray
        Index mask specifying which peaks are retained after precursor peak
        filtering
    """
    mask = (spectrum_mz<=np.absolute(precursor_mz-precursor_tolerance)) | (spectrum_mz>=np.absolute(precursor_mz+precursor_tolerance))
    return mask


def _select_top_intensity(spectrum_intensity: np.ndarray, max_num_peaks: int = 50):
        """
        Only the `max_num_peaks` most intense fragment peaks are retained.

        Parameters
        ----------
        max_num_peaks : Optional[int], optional
            Only retain the `max_num_peaks` most intense peaks (the default is
            None, which retains all peaks).

        Returns
        -------
        np.ndarray
            Index mask specifying which peaks are retained after intensity filtering
        """
        intensity_idx = np.argsort(spectrum_intensity)
        
        # Only retain at most the `max_num_peaks` most intense peaks.
        mask = np.full_like(spectrum_intensity, False, np.bool_)
        mask[intensity_idx[-min(len(intensity_idx), max_num_peaks):]] = True
        return mask
    
    
def _scale_intensity(spectrum_intensity: np.ndarray, scaling: str = None, max_intensity: float = None):
    """
        Scale the intensity of the fragment peaks.

        Two types of scaling can be performed: scaling all peaks using a
        specific transformation and scaling the peaks relative to the most
        intense peak.

        Parameters
        ----------
        scaling : {'root', 'log'}, optional
            Method to scale the peak intensities (the default is None, which
            means that no transformation will be performed).
            Potential transformation options are:
            
            - 'root': Root-transform the peak intensities. The default is a
              square root transformation (`degree` is 2). The degree of the
              root can be specified using the `degree` kwarg.
            - 'log':  Log-transform the peak intensities. The default is a log2
              transformation (`base` is 2) after summing the intensities with 1
              to avoid negative values after the transformation. The base of
              the logarithm can be specified using the `base` kwarg.

        max_intensity : Optional[float], optional
            Intensity of the most intense peak relative to which the peaks will
            be scaled (the default is None, which means that no scaling
            relative to the most intense peak will be performed).

        Returns
        -------
        self : `MsmsSpectrum`
    """
    if scaling == 'root':
        spectrum_intensity = np.sqrt(spectrum_intensity).astype(np.float32)
    elif scaling == 'log':
        base = 2
        spectrum_intensity = (np.log1p(spectrum_intensity) / np.log(base)).astype(np.float32)
    
    if max_intensity is not None:
        spectrum_intensity = spectrum_intensity * max_intensity / spectrum_intensity.max()
        
    return spectrum_intensity


def _norm_intensity(spectrum_intensity: np.ndarray, scale: float = 1.0):
    """
    Normalize cluster peak intensities by their vector norm.
    Parameters
    ----------
    spectrum_intensity : np.ndarray
        The cluster peak intensities to be normalized.
    Returns
    -------
    np.ndarray
        The normalized peak intensities.
    """
    return spectrum_intensity / spectrum_intensity.max() * scale

In [2]:
def get_spec_stat(spectra_dict):
    total_spec = 0
    print('Spectrum statistics:')
    for charge_i in spectra_dict.keys():
        print(charge_i, len(spectra_dict[charge_i]))
        total_spec += len(spectra_dict[charge_i])

    print('Total spectrum: ', total_spec)

## Quantizer for MS Preprocessing

In [3]:
class ms_quantizer():
    def __init__(self, convert_exp=5, convert_frac=10):
        self.convert_exp = convert_exp
        self.convert_frac = convert_frac
        
        self.fp32_frac_bit = 23
        self.fp32_exp_bit = 8

        self.fp64_frac_bit = 52
        self.fp64_exp_bit = 11

        # For fp32
        self.offset_fp32 = 127-2**(self.convert_exp-1)+1
        self.mask_fp32 = np.array(0).astype(np.int32)            
        for i in range(self.fp32_frac_bit-self.convert_frac, self.fp32_frac_bit+self.convert_exp, 1):
            self.mask_fp32 = (self.mask_fp32 | np.array(1).astype(np.int32)<<i).astype(np.int32)
        
        # For fp64
        self.offset_fp64 = 1023-2**(self.convert_exp-1)+1
        self.mask_fp64 = np.array(0).astype(np.int64)            
        for i in range(self.fp64_frac_bit-self.convert_frac, self.fp64_frac_bit+self.convert_exp, 1):
            self.mask_fp64 = (self.mask_fp64 | np.array(1).astype(np.int64)<<i).astype(np.int64)
    
    def quantize(self, num):
        if num.dtype == np.float32:
            num = num * (np.array(2**self.offset_fp32).astype(np.float32))
            conveted_num = np.bitwise_and(num.view(np.int32), self.mask_fp32)
            conveted_num = conveted_num.view(np.float32) * (2**self.offset_fp32)
        else:
            num = num * (np.array(2**self.offset_fp64).astype(np.float64))
            conveted_num = np.bitwise_and(num.view(np.int64), self.mask_fp64)
            conveted_num = conveted_num.view(np.float64) * (2**self.offset_fp64)
    
        return conveted_num.astype(num.dtype)

In [20]:

# mgf_path = '/home/workspace/MS-Tools/PXD001197/'
mgf_path = '/home/workspace/MS-Tools/PXD021328/'


mgf_file_list = glob.glob(mgf_path+'*.mgf')


# Using fp16 quantization
mz_exp_bit = 5
peak_exp_bit = 8

mz_frac_bit = 28-mz_exp_bit
peak_frac_bit = 24-peak_exp_bit

fp16_quantizer_mz = ms_quantizer(convert_exp=mz_exp_bit, convert_frac=mz_frac_bit)
fp16_quantizer_peak = ms_quantizer(convert_exp=peak_exp_bit, convert_frac=peak_frac_bit)

out_path = './json_output/'
out_file_name = "spectra_fp_mz_{}_{}_peak_{}_{}".format(mz_exp_bit, mz_frac_bit, peak_exp_bit, peak_frac_bit)
out_file_name = out_path + out_file_name

args = {
    'file_list': mgf_file_list, 
    'mz_min': 200, 
    'mz_max': 2000, 
    'min_mz_range': 250, 
    'min_peaks': 5, 
    'max_peaks_used': 50, 
    'remove_precursor_tol': 0.05, 
    'min_intensity': 0.01, 
    'scaling': 'log', 
    'out_file_name': out_file_name,

    'if_fp_quantize': True,
    'mz_exp_bit': mz_exp_bit,
    'mz_frac_bit': mz_frac_bit,
    'peak_exp_bit': peak_exp_bit,
    'peak_frac_bit': peak_frac_bit,
    'fp_quantizer_mz': fp16_quantizer_mz,
    'fp_quantizer_peak': fp16_quantizer_peak
}

processed_spectra_dict, spectra_len = run_baseline(**args)

{'file_list': ['/home/workspace/MS-Tools/PXD021328/SARS-CoV-2-18.mgf', '/home/workspace/MS-Tools/PXD021328/SARS-CoV-2-34.mgf', '/home/workspace/MS-Tools/PXD021328/SARS-CoV-2-38.mgf', '/home/workspace/MS-Tools/PXD021328/SARS-CoV-2-47.mgf', '/home/workspace/MS-Tools/PXD021328/SARS-CoV-2-51.mgf'], 'mz_min': 200, 'mz_max': 2000, 'min_mz_range': 250, 'min_peaks': 5, 'max_peaks_used': 50, 'remove_precursor_tol': 0.05, 'min_intensity': 0.01, 'scaling': 'log', 'out_file_name': './json_output/spectra_fp_mz_5_23_peak_8_16', 'if_fp_quantize': True, 'mz_exp_bit': 5, 'mz_frac_bit': 23, 'peak_exp_bit': 8, 'peak_frac_bit': 16, 'fp_quantizer_mz': <__main__.ms_quantizer object at 0x7fdea815d0a0>, 'fp_quantizer_peak': <__main__.ms_quantizer object at 0x7fdea81a9dc0>}
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   39.0s remaining:   58.5s
[Parallel(n_jobs=-1)]

In [ ]:
processed_spectra_dict

In [19]:
processed_spectra_dict['quantization_error'] # fp32 baseline

[4.822454185448864e-05, 5.3618281881513836e-08]

In [22]:
processed_spectra_dict['quantization_error'] # 5+23 - 8+16

[5.238321293576252e-05, 7.102931036443806e-08]

In [35]:
for mz_bit in range(32, 20, -1):
#     for peak_bit in range(29, 22, -1):
    for peak_exp_bit in range(6, 8, 1):
        # Using fp16 quantization
        mz_exp_bit = 5
        peak_bit = mz_bit-2
                
        mz_frac_bit = mz_bit-1-mz_exp_bit
        peak_frac_bit = peak_bit-1-peak_exp_bit

        fp16_quantizer_mz = ms_quantizer(convert_exp=mz_exp_bit, convert_frac=mz_frac_bit)
        fp16_quantizer_peak = ms_quantizer(convert_exp=peak_exp_bit, convert_frac=peak_frac_bit)

        out_path = './json_output/'
        out_file_name = "spectra_fp_mz_{}_{}_peak_{}_{}".format(mz_exp_bit, mz_frac_bit, peak_exp_bit, peak_frac_bit)
        out_file_name = out_path + out_file_name

        args = {
            'file_list': mgf_file_list, 
            'mz_min': 200, 
            'mz_max': 2000, 
            'min_mz_range': 250, 
            'min_peaks': 5, 
            'max_peaks_used': 50, 
            'remove_precursor_tol': 0.05, 
            'min_intensity': 0.01, 
            'scaling': 'log', 
            'out_file_name': out_file_name,

            'if_fp_quantize': True,
            'mz_exp_bit': mz_exp_bit,
            'mz_frac_bit': mz_frac_bit,
            'peak_exp_bit': peak_exp_bit,
            'peak_frac_bit': peak_frac_bit,
            'fp_quantizer_mz': fp16_quantizer_mz,
            'fp_quantizer_peak': fp16_quantizer_peak
        }

        processed_spectra_dict, spectra_len = run_baseline(**args)

        print("mz:{}_{}\t intensity:{}_{}\t error{}".format(mz_exp_bit, mz_frac_bit, peak_exp_bit, peak_frac_bit, processed_spectra_dict['quantization_error']))

mz:5_26	 intensity:6_23	 error[8.120657014452185e-06, 4.577550140936715e-09]
mz:5_26	 intensity:7_22	 error[8.472392486174613e-06, 5.617242178166919e-09]
mz:5_25	 intensity:6_22	 error[1.323469309894031e-05, 9.523837932292356e-09]
mz:5_25	 intensity:7_21	 error[1.3706470184666176e-05, 1.0489876783164584e-08]
mz:5_24	 intensity:6_21	 error[2.125473256040862e-05, 1.8409767664008334e-08]
mz:5_24	 intensity:7_20	 error[2.125473256040862e-05, 1.8606088424106233e-08]
mz:5_23	 intensity:6_20	 error[4.904805399718361e-05, 5.582031970110382e-08]
mz:5_23	 intensity:7_19	 error[4.941690459830998e-05, 5.78370422186584e-08]
mz:5_22	 intensity:6_19	 error[7.900403139013153e-05, 9.00967232392305e-08]
mz:5_22	 intensity:7_18	 error[7.9627764156364e-05, 9.18300412136077e-08]
mz:5_21	 intensity:6_18	 error[0.00015654118035393364, 2.084433509915016e-07]
mz:5_21	 intensity:7_17	 error[0.00015680378143892822, 2.105982261086578e-07]
mz:5_20	 intensity:6_17	 error[0.000329677931234894, 3.981636227559744e-07]

In [ ]:
processed_spectra_dict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})


import os
import logging
import pyteomics.mgf
from joblib import Parallel, delayed

In [ ]:
logger = logging.getLogger('ms_preprocess_fp')

def run_spect_stat(mgf_file_list):
    
    results = Parallel(n_jobs=-1, verbose=100)(delayed(get_spect_stat)(file_i) for file_i in mgf_file_list)
    
    spectra_len = np.array([])
    spectra_mz = np.array([])
    spectra_peak = np.array([])
    
    for r_i in results:        
        spectra_len = np.append(spectra_len, r_i[0])
        spectra_mz = np.append(spectra_mz, r_i[1])
        spectra_peak = np.append(spectra_peak, r_i[2])
        
    return spectra_len, spectra_mz, spectra_peak
    
            
def get_spect_stat(mgf_file):
    f_in = pyteomics.mgf.MGF(mgf_file, convert_arrays=1)
    filename = os.path.splitext(os.path.basename(f_in.name))[0]
    logger.info('Processing mgf file: %s', filename)
    
    spectra_len = np.array([])
    spectra_mz = np.array([])
    spectra_peak = np.array([])
    
    
    for spectrum_i, spectrum_dict in enumerate(f_in):
        spec_i_mz = spectrum_dict['m/z array'].astype(np.float32)
        spec_i_intensity = spectrum_dict['intensity array'].astype(np.float32)

        spectra_len = np.append(spectra_len, len(spec_i_mz))
#         spectra_mz = np.append(spectra_mz, spec_i_mz)
#         spectra_peak = np.append(spectra_peak, spec_i_intensity)
        
    return spectra_len, spectra_mz, spectra_peak 


In [ ]:

import glob

mgf_path = '/home/workspace/MS-Tools/PXD021328/'
# mgf_path = '/home/workspace/MS-Tools/PXD001197/'
mgf_file_list = glob.glob(mgf_path+'*.mgf')

# mgf_file_list = [
# '/home/workspace/MS-Tools/PXD001197/OrbiElite04550.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04551.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04552.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04553.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04554.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04555.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04556.mgf',
#  '/home/workspace/MS-Tools/PXD001197/OrbiElite04557.mgf']

print(mgf_file_list)

results = run_spect_stat(mgf_file_list)




In [ ]:
# Plot Histogram on Spectra Length
plt.hist(results[0], density=True, bins=500)
plt.gca().set(title='Spectra Length', ylabel='Frequency');

In [ ]:
# Plot Histogram on m/z Distribution
plt.hist(results[1], density=True, bins=500)
plt.gca().set(title='m/z Distribution', ylabel='Frequency');

In [ ]:
# Plot Histogram on peak Distribution
plt.hist(results[2], density=True, bins=500)
plt.gca().set(title='Peak Itensity Distribution', ylabel='Frequency')
plt.xlim([0, 50000])